In [526]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re
import pandas as pd
from sklearn.model_selection import train_test_split

# Generating Card Games

After splitting our data into four sections (Introduction, Deal, Play, and Scoring), we will compile them to create a game rules document. We will be applying a recurrent neural network and LTSM to achieve this. 

## Preprocessing
First, we will import and clean our data.

In [527]:
data = pd.read_csv('../data/text_data_grouped_by_cat.csv').drop('Unnamed: 0', axis=1).drop(8091).drop_duplicates()

# Clean data
data['Text'] = (data['Text']
                   .apply(lambda x: x.lower()
                                      .replace('\n', '')))

In [528]:
# Create subsets
introductions = data.loc[data['index'] == 'Introduction']
deal = data.loc[data['index'] == 'Deal']
play = data.loc[data['index'] == 'Play']
scoring = data.loc[data['index'] == 'Scoring']

## Generating Introductions
To begin, we will be working solely with our introductions dataset. We are referencing [this notebook](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_10_3_text_generation.ipynb), which references the keras documentation.

In [580]:
# define parameters
maxlen = 100
step = 3
BATCH_SIZE = 64
epochs = 60

## Preprocessing
First, we create a function to process the text. For the sake of this project, we are only going to keep ascii characters.

In [609]:
def process_text(df):   
    string = ''
    for i in df["Text"]:
        string+=i
    # keep only ascii
    return re.sub(r'[^\x00-\x7f]',r'', string)

In [629]:
processed_text = process_text(deal)

## Setup
Next, we are going to create a set up function. The function performs the following actions:
1. create a dictionary to map characters to numbers
2. divide our text into sample sequences to train our model on
3. vectorize our sequences into matrix form

In [630]:
def setup(processed_text, maxlen, step):
    
    # create dictionary
    chars = sorted(list(set(processed_text)))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))
    
    # divides text into sample sequences
    sentences = []
    next_chars = []
    for i in range(0, len(processed_text) - maxlen, step):
        sentences.append(processed_text[i: i + maxlen])
        next_chars.append(processed_text[i + maxlen])
    
    # vectorize into matrix
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test, sentences, char_indices, indices_char, chars

In [632]:
# Split data into train/test
X_train, X_test, y_train, y_test, sentences, char_indices, indices_char, chars = setup(processed_text, maxlen, step)

## Build Model
We now are ready to build our model. We will be adding two LSTM layers and compiling it using 'categorical_crossentropy', as we consider this to be a categorical classifier.

In [633]:
model = Sequential()
model.add(LSTM(BATCH_SIZE, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(BATCH_SIZE))
model.add(Dense(len(chars), activation='softmax'))
#model.add(tf.keras.layers.Dropout(0.4))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [634]:
model.summary()

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 100, 64)           29696     
_________________________________________________________________
lstm_53 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_40 (Dense)             (None, 51)                3315      
Total params: 66,035
Trainable params: 66,035
Non-trainable params: 0
_________________________________________________________________


## Train
Lastly, we train our model. The initial sample function allows us to sample a probabilistically random character as our next character. Next, we will display the trained model with temperatures [0.2, 0.5, 1.0, 1.2].

In [635]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [636]:
def on_epoch_end(epoch, _):
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
    print('\nhistory dict:', model.history)


In [637]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=21)

model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=epochs,
          callbacks=[print_callback, es_callback],
          validation_data=(X_test, y_test))

Epoch 1/60
244/244 [==============================] - ETA: 0s - loss: 2.3544****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "he dealer's right, until everyone has a hand of nine cards. the dealer places the last four cards fa"
he dealer's right, until everyone has a hand of nine cards. the dealer places the last four cards face cards are the the cards are the the the the cards the the the the the the cards the the the the cards the the the cards the the cards the the the the cards the the the cards and the the the cards the cards the the the the cards the the cards the the the the the the cards and the player the the the cards the cards and the the the the cards the the the the the the the the cards the the the cards 
----- temperature: 0.5
----- Generating with seed: "he dealer's right, until everyone has a hand of nine cards. the dealer places the last four cards fa"
h

pondingly fewer cards are dealt to this player, so that everyone begins the play with 4 cards. when a srbpalde shore cards righ 2y the lrave tex mituled ,e andis gath if of thesre atimiind willeb2xill frer the yen's atible becore firse, the vex )beacosent at the tiller are ro mealer it. the cards as deal. are cares  facedy player of and 4pie poco pore mas vowre uus thrie cards inwllhand. the cuts oned, as thl dece in the sokt. the it sale sumteal, yo . the cut for es  face--b 6 pace de1k. three 

history dict: <tensorflow.python.keras.callbacks.History object at 0x2472ee410>
244/244 [==============================] - 70s 286ms/step - loss: 1.4229 - val_loss: 1.4056
Epoch 4/60
244/244 [==============================] - ETA: 0s - loss: 1.2644****************************************************************************
----- Generating text after Epoch: 3
----- temperature: 0.2
----- Generating with seed: "s chosen by a random method: for example all draw cards and the lowest deals. the tu

, and the remaining undealt cards are placed face down next to it to form the draw pile or stock. dutf three cards, this cards are deals. if at a siin, peacs in coss conting. if the cards are duaks sibpuate, is the rounder of the pliyed the deal in a card betines dealt the sos  a backel gapeer of the finst to e-cards are shuffle msuen to the cards all muns the cards are player left. nardey of the nust from cardr ace disem (0 to four cards are mone of three cards in destat un one ote, in as stive
----- temperature: 1.2
----- Generating with seed: ", and the remaining undealt cards are placed face down next to it to form the draw pile or stock. du"
, and the remaining undealt cards are placed face down next to it to form the draw pile or stock. dufled in in cut asith amay fot 6cides excrtition) each tree. the two cards anter all wh. each, bal p) the deals of coors. the nok of hasss. after the hast dealel rethe deacer undly with of threep, will desef, will seed. the 5 card may dealersall 

t is noticed. the announcement is proved by rapidly dealing the cards face up onto the table after youndly up ontil cards to each player of the player is chosen by and wide the player to dealer's left. ithe dealer is chosen by and playeds of the player is chosen by dealt a the player to dealer's rounding of three cards and of the table to begins played the player to dealer's right face down in the player is chosen by and played the player is clockwise. the player to dealer's right. after each ha
----- temperature: 1.0
----- Generating with seed: "t is noticed. the announcement is proved by rapidly dealing the cards face up onto the table after y"
t is noticed. the announcement is proved by rapidly dealing the cards face up onto the table after youndly up and place the vass .tle player game, deals the player to hem be any playeracas as placed all  of three player with the stock pleiigating wo fourmed of the fare deals the player to deal and placed be the player the top  of erst tree car

ith the dealer's other five cards when everyone has had a chance to look at it. the players pick up the pecer of the player to dealer's left and gan but a paribe. this is a player to dealer's left and ganhed the turn to deal passes to the player to dealer's left and gan but a parible in the pack and players left a player to dealer's left ecoven cominds. the cards are dealt to the players left and there are dealt to the player to dealer's left and ganhed becomen that the cards are dealt to the pl
----- temperature: 0.5
----- Generating with seed: "ith the dealer's other five cards when everyone has had a chance to look at it. the players pick up "
ith the dealer's other five cards when everyone has had a chance to look at it. the players pick up cards. the cards are dealt to the player to dealer's left. ither all the cards are placed face down to dealer turns the dealer deals the cards are dealt on the table, and the dealer is chosen by drawny counted and there are dealt to the player t

...

he cards have been played and scored the turn to deal passes to the left. the cards are shuffled by the dealer deals them for the deal and the dealer deals then the deal and deal foun this players left a player cards the tor thid face down to the tom the cut and place trom the to form a to shuffled loles enting   coreader in the first three cards face down to the reft and players will pile, there are four cards face down to the dealer deals one four cards in a time, fron the top three are trump 
----- temperature: 1.0
----- Generating with seed: "he cards have been played and scored the turn to deal passes to the left. the cards are shuffled by "
he cards have been played and scored the turn to deal passes to the left. the cards are shuffled by the dealer istats by the player to dealer's left. this play is not recat sen is thet a sinder of the table, by the first three, ack of 8 cards incont a time, fnon and dealt neme of s altionals equr. thit the right-and the dealerd is dealt a deal

d of 18 cards, normally either in batches of 2, going 9 times around the table, or in batches of 3, gormo them is a time and dealt one at a time and there are stacked face down in the cess as i sistathion. the player to the player to the stock. the player to the player who cards face up on the table, stackiplals and the plal is an eich has thin in a time of as there are stacked face down, startith the player who can by dealt to each player and three cards are stacked face down in the middle in t
----- temperature: 0.5
----- Generating with seed: "d of 18 cards, normally either in batches of 2, going 9 times around the table, or in batches of 3, "
d of 18 cards, normally either in batches of 2, going 9 times around the table, or in batches of 3, gorve that everying play who has dealt to each player, then dealt to each player and two cards there cart the cut and set one face up and the player the deal is cut and descart on the cards are dealt to each player and preving.  efore seven on t

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


ay dealt to each player then dealt who has batcos mot of three cards, and are draw clockwise on the table. the next (uftion hands. the table. then this players mock i sas bottom (mons cards. thrte this is the , iver for four.  car
----- temperature: 1.2
----- Generating with seed: "d of 18 cards, normally either in batches of 2, going 9 times around the table, or in batches of 3, "
d of 18 cards, normally either in batches of 2, going 9 times around the table, or in batches of 3, gormo who first..  the cut and startscer has 1n is (gimp this play are cards each   wild cards to each player, one at a time, clockwise. the lifts sorti1: opfouly card the next homded, and fis mist.  set jove will card e face down, stourt lift has but a two 6 "ffareing.  wiehle). after the middle if clrcaras an the next. thing. this givied clockwision to for exampling, or this in one entle fidad m

history dict: <tensorflow.python.keras.callbacks.History object at 0x243d20310>
244/244 [========================

other three cards to each player, but no further face up cards are dealt to the table. this is repeated by the dealer shuffles the next have face up and dealt too a ponnnoe of each face up cards theron at a time, fore. cards to the number on to the left and games face up on the table to begin the cards are placed face underown the cards, and the player to the left after each hame the player to the left a card to the next player and bephinith the cards are placed bechiss the next player.  the num
----- temperature: 1.0
----- Generating with seed: "other three cards to each player, but no further face up cards are dealt to the table. this is repea"
other three cards to each player, but no further face up cards are dealt to the table. this is repeated party the dealer shuffles, then deals a bisck to co startle the left after each hame. the lole preinis dealt to each player, then 1 haml be dealts who hand of the pealrs  more.  have ive fourn (mourde sgath. the lole moos may bat a time anti

In [628]:
model.predict(X_test, batch_size=BATCH_SIZE, callbacks=[print_callback])

array([[1.22649758e-03, 1.44530937e-12, 5.64427390e-23, ...,
        9.88045752e-01, 1.05654214e-08, 3.68800421e-08],
       [2.63757654e-03, 8.35858889e-14, 3.92836886e-13, ...,
        2.13889635e-08, 3.70670818e-08, 1.25211697e-09],
       [1.13751639e-05, 1.13163365e-14, 5.59334183e-16, ...,
        1.07701033e-15, 8.39388700e-29, 2.23236568e-10],
       ...,
       [2.72701420e-02, 3.13624793e-10, 1.26710336e-06, ...,
        6.83269377e-07, 1.68594767e-07, 5.19585308e-09],
       [2.26948949e-04, 5.02599018e-10, 7.12832789e-30, ...,
        5.57322737e-32, 1.12464847e-13, 3.22807608e-07],
       [2.12705117e-02, 1.33456332e-12, 3.71563528e-03, ...,
        3.47890978e-04, 7.70646101e-03, 1.67013297e-07]], dtype=float32)